## Setup

In [ ]:
import tio4550_housing_energy.utils as u

# This will now work!
u.a_test_function()

# Test of OpenAI's Responses API

In [ ]:
import os
import pandas as pd
import json
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 1. SETUP & AUTHENTICATION ---
# Load the environment variable for the API key from a .env file.
load_dotenv()

# Instantiate the OpenAI client.
# It will automatically use the OPENAI_API_KEY from your .env file.
try:
    client = OpenAI()
    if not os.getenv("OPENAI_API_KEY"):
        raise ValueError("OPENAI_API_KEY environment variable not found.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    print(
        "Please ensure your OPENAI_API_KEY is set correctly as an environment variable."
    )
    exit()

# --- 2. PROMPT ---
system_prompt = """
Du er en svært nøyaktig og detaljorientert ekspert på analyse av norske boligannonser. Oppgaven din er å identifisere og trekke ut relevant informasjon om variabler fra boligannonser basert på spesifikke instrukser og returnere det som et JSON objekt. 

**Generelle regler:**
1. Hvis en egenskap ikke er nevnt, settes dens _bool-verdi til 0 og dens _år-verdi til 0. 
2. Nevnes en egenskap uten tilhørende årstall, settes dens _bool-verdi til 1 og _år til 0. 
3. Nevnes både egenskap og årstall, settes _bool til 1 og _år til oppgitt årstall. 
4. Ved flere årstall for samme egenskap, bruk det nyeste. 
5. Ditt endelige resultat MÅ være et enkelt, gyldig JSON-objekt med KUN nøklene spesifisert nedenfor. 

**Spesifikke regler:**
1. ot_bool (Totalrenovering): Sett til 1 KUN "totalrenovering” er beskrevet i detalj. Ignorer generelle eller vage fraser som "omfattende renovert" dersom de kun beskriver kosmetiske endringer. Da skal ot_bool være 0. 
2. t_bool (Tak): Sett til 1 kun ved konkret bytte eller nytt tak (eksempel: "nytt tak"). Ikke vedlikehold, inspeksjon eller rens. 
3. k_bool (Kledning): Sett til 1 kun for bytte av ytre veggpanel (eksempel: "ny kledning"). Maling teller ikke. 
4. e_bool (Etterisolering): Sett til 1 ved forbedring av isolasjon (eksempel: "etterisolert"). Bytte av vinduer, kledning eller gulv telles ikke. 
5. v_bool (Vinduer): Sett til 1 hvis vinduer er byttet (eksempel: "nye vinduer"). Bytte av kun glass eller maling av karm teller ikke. 
6. ob_bool (Bad), ok_bool (Kjøkken), og_bool (Gulv): Sett til 1 hvis det er nevnt nytt, oppgradert eller byttet. 
7. vp_bool (Varmepumpe): Sett til 1 hvis installasjon av varmepumpe er nevnt. 

**JSON Schema:**
- `vp_bool`, `vp_år`: (Indikerer om varmepumpe er installert/årstall) 
- `e_bool`, `e_år`: (Etterisolering gjennomført/årstall) 
- `v_bool`, `v_år`: (Nye eller oppgraderte vinduer/årstall) 
- `k_bool`, `k_år`: (Ny kledning på yttervegger/årstall) 
- `t_bool`, `t_år`: (Tak byttet eller oppgradert/årstall) 
- `ot_bool`, `ot_år`: (Totalrenovering er gjennomført/årstall) 
- `ob_bool`, `ob_år`: (Oppgradering av bad/årstall) 
- `ok_bool`, `ok_år`: (Oppgradering av kjøkken/årstall) 
- `og_bool`, `og_år`: (Oppgradering av gulv/årstall) 
"""


# --- 3. LOAD DATASET ---
input_data_path = "../data/2_interim/relevant_ads_sampled.csv"
try:
    df = pd.read_csv(input_data_path, delimiter=";")
    print(f"Successfully loaded {len(df)} ads from {input_data_path}")
except FileNotFoundError:
    print(f"Error: The file was not found at {input_data_path}")
    exit()


# --- 4. DEFINE API CALL FUNCTION ---
def analyze_ad_description(description):
    """Calls the OpenAI API. Includes robust error handling."""
    if not isinstance(description, str) or not description.strip():
        return {}  # Return an empty dict for invalid input
    try:
        response = client.responses.create(
            model="gpt-5-nano",
            reasoning={"effort": "medium"},
            # temperature=0.1, TODO Find a parameter that controls temperature. Apparently, "Temperature" dosent work right now.
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": description},
            ],
            text={
                "format": {"type": "json_object"},
            },
        )
        json_output = response.output_text
        return json.loads(json_output)
    except json.JSONDecodeError:
        print("Warning: Failed to parse JSON from API response for one ad.")
        return {}  # Return empty dict if JSON is malformed
    except Exception as e:
        print(f"An API error occurred: {e}")
        return {}  # Return empty dict on API failure


# --- 5. PROCESS THE DATAFRAME IN PARALLEL ---
descriptions = df["description"].tolist()
api_results = [{} for _ in descriptions]  # Initialize with empty dicts

with ThreadPoolExecutor(
    max_workers=5
) as executor:  # Increased workers slightly for speed
    future_to_idx = {
        executor.submit(analyze_ad_description, desc): idx
        for idx, desc in enumerate(descriptions)
    }
    for future in tqdm(
        as_completed(future_to_idx), total=len(descriptions), desc="Analyzing Ads"
    ):
        idx = future_to_idx[future]
        try:
            api_results[idx] = future.result()
        except Exception as e:
            print(f"Error processing result for index {idx}: {e}")
            api_results[
                idx
            ] = {}  # Ensure it's an empty dict on any future-related error

# --- 6. ROBUSTLY PARSE RESULTS AND SAVE ---
# Define all the columns you expect to get from the API
expected_columns = [
    "vp_bool",
    "vp_år",
    "el_bool",
    "el_år",
    "e_bool",
    "e_år",
    "v_bool",
    "v_år",
    "k_bool",
    "k_år",
    "t_bool",
    "t_år",
    "s_bool",
    "s_år",
    "ot_bool",
    "ot_år",
    "ob_bool",
    "ob_år",
    "ok_bool",
    "ok_år",
    "og_bool",
    "og_år",
]

# Create a clean list of dictionaries, ensuring every dictionary has all expected keys
clean_results = []
for result in api_results:
    clean_dict = {}
    for col in expected_columns:
        # Use .get() with a default value of 0 to handle missing keys gracefully
        clean_dict[col] = result.get(col, 0)
    clean_results.append(clean_dict)

# Create the results DataFrame from the clean list
results_df = pd.DataFrame(clean_results)

# Combine the original DataFrame with the new results DataFrame
final_df = pd.concat([df, results_df], axis=1)

# --- THIS IS THE NEW STEP ---
# Drop the original text columns as they are no longer needed for the final dataset
final_df = final_df.drop(columns=["description", "title"])

# Define the output path
output_path = "../data/3_processed/analyzed_ads_data_clean.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save the final DataFrame
final_df.to_csv(output_path, index=False)

print(f"\n✅ Processing complete! Clean data saved to {output_path}")
print("\nHere's a preview of the clean final data (without description and title):")
print(final_df.head())

Error: The file was not found at ../data/2_interim/relevant_ads_sampled_for_chatgpt.csv


NameError: name 'df' is not defined

: 

In [ ]:
final_df.head()